## Homework: Vector Search

In this homework, we'll experiemnt with vector with and without Elasticsearch


In [1]:
## Q1. Getting the embeddings model
from sentence_transformers import SentenceTransformer
model_name="multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
user_question = "I just discovered the course. Can I still join it?"
embedding_model.encode(user_question)[0]

np.float32(0.078222655)

What's the first value of the resulting vector?
* 0.07


In [3]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [4]:
type(documents)

list

In [18]:
filtered_documents =[]
for doc in documents:
    if doc["course"]=="machine-learning-zoomcamp":
        filtered_documents+=[doc]  
documents = filtered_documents  
len(filtered_documents)

375

In [19]:
## Q2. Creating the embeddings
from tqdm import tqdm
embeddings=[]
for doc in tqdm(documents):
    qa_text = f'{doc["question"]} {doc["text"]}'
    embeddings+=[embedding_model.encode(qa_text)]


100%|██████████| 375/375 [01:06<00:00,  5.62it/s]


In [20]:
import numpy as np
X = np.array(embeddings)
print(X.shape)

(375, 768)


What's the shape of X? (`X.shape`). Include the parantheses. 

(375, 768)

## Q3. Search


In [40]:
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)
scores = X.dot(v)
max(scores)

np.float32(0.6506573)

In [22]:
scores

array([ 2.89217263e-01,  4.35050726e-01,  2.20572561e-01,  1.28085926e-01,
        3.28754455e-01,  4.58999664e-01,  4.48930591e-01,  2.70170599e-01,
        2.13975310e-01,  3.39745760e-01,  1.63339168e-01,  3.15918088e-01,
        1.53621323e-02,  4.14240420e-01,  6.50657296e-01,  4.12786484e-01,
        2.02390462e-01,  4.71707582e-01,  3.74821007e-01,  3.53319108e-01,
        4.50558245e-01,  1.85430944e-01,  3.45526993e-01,  8.22948217e-02,
       -2.19077598e-02, -4.64250743e-02,  8.60690773e-02,  9.87197682e-02,
       -2.14188285e-02,  1.28751006e-02,  2.54870541e-02,  3.66866291e-02,
        4.56338078e-02, -2.05279887e-02,  1.82662532e-02,  3.39789167e-02,
        1.66510604e-03,  1.03209512e-02,  3.40170749e-02, -2.65231431e-02,
       -4.11909968e-02, -4.97983992e-02, -6.20346665e-02, -7.63561390e-03,
        4.61616889e-02,  6.95944112e-03,  4.77613509e-02, -9.97766852e-04,
       -5.60126789e-02, -5.96206151e-02,  2.05209196e-01, -1.16968602e-01,
       -2.34160442e-02, -

In [23]:
scores.argmax()


np.int64(14)

What's the highest score in the results?
- 0.65

In [24]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)


[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

## Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will
use the hitrate metric for evaluation.

In [25]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [26]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)
 
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)



In [27]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)
    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }


In [28]:
def cosine_search(q):
    question = q['question']

    v_q = embedding_model.encode(question)
    return search_engine.search(v_q, num_results=5)

In [29]:
evaluate(ground_truth, cosine_search)

  0%|          | 0/1830 [00:00<?, ?it/s]

100%|██████████| 1830/1830 [01:19<00:00, 23.08it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

What did you get?

* 0.93


## Q5. Indexing with Elasticsearch


In [43]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}


In [44]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [52]:
from tqdm.auto import tqdm
for doc,em in zip(tqdm(documents),embeddings):
    doc['question_text_vector'] = em


100%|██████████| 375/375 [00:00<00:00, 187268.01it/s]


In [53]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:07<00:00, 47.96it/s]


In [55]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [56]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = embedding_model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

In [62]:
query = 'I just discovered the course. Can I still join it?'
v_q = embedding_model.encode(query)
elastic_search_knn('question_text_vector', v_q, "machine-learning-zoomcamp")

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the

What's the ID of the document with the highest score?
ee58a693

## Q6. Hit-rate for Elasticsearch

The search engine we used in Q4 computed the similarity between
the query and ALL the vectors in our database. Usually this is 
not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster. 

Let's evaluate how worse the results are when we switch from
exact search (as in Q4) to approximate search with Elastic.

What's hitrate for our dataset for Elastic?

* 0.93

In [61]:
evaluate(ground_truth, question_vector_knn)

100%|██████████| 1830/1830 [01:33<00:00, 19.64it/s]


{'hit_rate': 0.9103825136612022, 'mrr': 1.2421038251366117}